In [46]:
from google.cloud import logging
import datetime
import json
import pandas as pd

from importlib import reload


import src.log_queries as queries
import src.transforms as transforms

reload(queries)
reload(transforms)

from src.log_queries import (
    CREATE_PODS,
    SCALE_UP_DECISION,
    NODE_READY,
    NODE_READY_PATCH,
    POD_SCHEDULED,
    CONTAINER_STARTED_PATCH,
    CONTAINER_STARTED,
    DELETE_PODS,
    SCALE_DOWN_DECISION,
    NODE_CORDONED,
    NODE_NOT_READY,
    NODE_DELETED,
    NODES_CREATE,
    NODES_DELETE
)

from src.transforms import (
    AdvancedDotDict,
    create_pods as create_pods_transform,
    container_started as container_started_transform,
    pod_scheduled as pod_scheduled_transform,
    nodes_delete as nodes_delete_transform,
    nodes_create as nodes_create_transform,
    node_ready_patch as node_ready_patch_transform
)
# Create a logging client
client = logging.Client()

PROJECT_ID = "symphony-dev-2"
TARGET_CLUSTER = "cluster-test-0"


# Set maximum width for table view
pd.set_option('max_colwidth', 60)
# Set maximum rows for table view
pd.set_option('display.max_rows',200)


### Template queries

In [21]:
TIME_DELTA_MINUTES = 20
START_TIME_STRING = "2025-08-26T09:16:15+00:00"
EXPERIMENT_TIME = datetime.datetime.fromisoformat(START_TIME_STRING).astimezone(datetime.UTC)
START_WINDOW = EXPERIMENT_TIME.isoformat(timespec="seconds")
# START_WINDOW = (EXPERIMENT_TIME - datetime.timedelta(minutes=TIME_DELTA_MINUTES)).isoformat(timespec="seconds")
STOP_WINDOW = (EXPERIMENT_TIME + datetime.timedelta(minutes=TIME_DELTA_MINUTES)).isoformat(timespec="seconds")

START_WINDOW

# Count validated
create_pods = CREATE_PODS.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# TODO: Validate detail count
scale_up_decision = SCALE_UP_DECISION.format(
    PROJECT_ID=PROJECT_ID,
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# TODO: Fix additional entries
node_ready = NODE_READY.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

node_ready_patch = NODE_READY_PATCH.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# TODO: Fix missing entries
pod_scheduled = POD_SCHEDULED.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# NOT USED
container_started_patch = CONTAINER_STARTED_PATCH.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# Count validated
container_started = CONTAINER_STARTED.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# Count validated
delete_pods = DELETE_PODS.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# TODO: Validate detail count
scale_down_decision = SCALE_DOWN_DECISION.format(
    PROJECT_ID=PROJECT_ID,
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# Count validated
node_cordoned = NODE_CORDONED.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# TODO: Fix extra counts
node_not_ready = NODE_NOT_READY.format(
CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

# Count validated
node_deleted = NODE_DELETED.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

nodes_create = NODES_CREATE.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)

nodes_delete = NODES_DELETE.format(
    CLUSTER_NAME=TARGET_CLUSTER,
    START_WINDOW=START_WINDOW,
    STOP_WINDOW=STOP_WINDOW
)


### Query

In [44]:
logs = []

for entry in client.list_entries(
    filter_=node_ready_patch,
    order_by=logging.DESCENDING,
    max_results=1000
): 
    logs.append(entry)    


print(len(logs))

635


In [47]:
data = []

try:
    for idx, log in enumerate(logs):
        data.append(node_ready_patch_transform(log.to_api_repr()))
except AttributeError as e:
    print(f"AttributeError at index {idx}")
    raise e

df = pd.DataFrame.from_records(data)

print(len(df.index))

635


625